In [24]:
import pandas as pd
import numpy as np

from sklearn.svm import SVR
import torch

import torch.nn as nn

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import GridSearchCV # to find hgyper params for svr

In [10]:
# 1)	unit number
# 2)	time, in cycles
# 3)	operational setting 1
# 4)	operational setting 2
# 5)	operational setting 3
# 6)	sensor measurement  1
# 7)	sensor measurement  2
# ...
# 26)	sensor measurement  26

cols = ['unit_number', 'cycles'] + [f'op_setting_{i}' for i in range(1, 4)] + [f'sensor_{i}' for i in range(1, 22)]
train_df = pd.read_csv('data/CMAPSS/train_FD001.txt', sep=' ', header=None)
train_df = train_df.dropna(axis=1, how='all')
train_df.columns = cols
rul_train = train_df.groupby('unit_number')['cycles'].max().reset_index()
rul_train.columns = ['unit_number', 'max_cycles']
train_df = train_df.merge(rul_train, on='unit_number')
train_df['RUL'] = train_df['max_cycles'] - train_df['cycles']


test_df = pd.read_csv('data/CMAPSS/test_FD001.txt', sep=' ', header=None)
test_df = test_df.dropna(axis=1, how='all')
test_df.columns = cols
true_rul = pd.read_csv('data/CMAPSS/RUL_FD001.txt', header=None)

In [ ]:
X_train = train_df[[f'sensor_{i}' for i in range(1, 22)]].values
y_train = train_df['RUL'].values

X_test = test_df[[f'sensor_{i}' for i in range(1, 22)]].values
y_test = np.repeat(true_rul[0].values, test_df.groupby('unit_number').size().values)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


svr = SVR(C =10, epsilon= 0.5, kernel= 'rbf')

svr.fit(X_train_scaled, y_train)




pred = svr.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, pred))
print(f"test RMSE: {rmse:.2f}")

Test RMSE: 80.30


In [66]:
class LSTM(nn.Module):
    def __init__(self, input_size=21, hidden_size=128, num_layers=5, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers,
            batch_first=True, dropout=dropout, bidirectional=True
        )
        self.fc = nn.Linear(hidden_size * 2, 1)  # bidirectional

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :] 
        out = self.fc(out)
        return out.squeeze()

In [98]:
sequence_length = 100
features = [f'sensor_{i}' for i in range(1, 22)]


X_list, y_list = [], []
for eid in train_df['unit_number'].unique():
    unit = train_df[train_df['unit_number'] == eid]
    data = unit[features].values
    rul = unit['RUL'].values
    for i in range(len(data) - sequence_length + 1):
        X_list.append(data[i:i+sequence_length])
        y_list.append(rul[i + sequence_length - 1])


X_tensor = torch.tensor(np.array(X_list), dtype=torch.float32)
y_tensor = torch.tensor(np.array(y_list), dtype=torch.float32)


dataset = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

print(f"data shape: {X_tensor.shape}, targets shape: {y_tensor.shape}")

data shape: torch.Size([10731, 100, 21]), targets shape: torch.Size([10731])


In [95]:
def rmse_loss(pred, target):
    return torch.sqrt(nn.functional.mse_loss(pred, target))



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTM().to(device)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=5e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

epochs = 100
best_loss = float('inf')
patience = 5
counter = 0



for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        #loss = rmse_loss(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(loader)
    scheduler.step(avg_loss)
    print(f"Epoch [{epoch+1:>3}/{epochs}] ({counter}/{patience}) loss: {avg_loss:.4f}")


    if avg_loss < best_loss:
        best_loss = avg_loss
        counter = 0
        best_model_state = model.state_dict()
    else:
        counter += 1
        if counter >= patience:
            print("early stop!")
            break


model.load_state_dict(best_model_state)

Epoch [  1/100] (0/5) loss: 3813.9562
Epoch [  2/100] (0/5) loss: 2410.7772
Epoch [  3/100] (0/5) loss: 2243.4498
Epoch [  4/100] (0/5) loss: 2235.4878
Epoch [  5/100] (0/5) loss: 2236.9574
Epoch [  6/100] (1/5) loss: 2234.9336
Epoch [  7/100] (0/5) loss: 2233.4643
Epoch [  8/100] (0/5) loss: 2234.1578
Epoch [  9/100] (1/5) loss: 2234.1558
Epoch [ 10/100] (2/5) loss: 2234.0658
Epoch [ 11/100] (3/5) loss: 2233.7287
Epoch [ 12/100] (4/5) loss: 2234.7090
early stop!


<All keys matched successfully>

In [99]:
X_test_list, y_test_list = [], []

for eid in test_df['unit_number'].unique():
    unit = test_df[test_df['unit_number'] == eid]
    data = unit[features].values
    rul_value = true_rul.iloc[eid - 1, 0]
    for i in range(len(data) - sequence_length + 1):
        X_test_list.append(data[i:i + sequence_length])
        y_test_list.append(rul_value)


X_test_tensor = torch.tensor(np.array(X_test_list), dtype=torch.float32)
y_test_tensor = torch.tensor(np.array(y_test_list), dtype=torch.float32)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model.eval()
all_preds, all_targets = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        preds = model(X_batch).cpu()
        all_preds.append(preds)
        all_targets.append(y_batch)

all_preds = torch.cat(all_preds)
all_targets = torch.cat(all_targets)
rmse = torch.sqrt(nn.functional.mse_loss(all_preds, all_targets))
print(f"test RMSE: {rmse:.2f}")

test RMSE: 39.89
